In [13]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import functools
import adanet
import tensorflow as tf

# The random seed to use
RANDOM_SEED = 42

In [14]:
(x_train, y_train), (x_test, y_test) = (tf.keras.datasets.boston_housing.load_data())

In [15]:
FEATURES_KEY = 'x'

def input_fn(partition, training, batch_size):
    # generate an inout function for the estimator
    def _input_fn():
        if partition == 'train':
            dataset = tf.data.Dataset.from_tensor_slices(({FEATURES_KEY: tf.log1p(x_train)}, tf.log1p(y_train)))
        else:
            dataset = tf.data.Dataset.from_tensor_slices(({FEATURES_KEY: tf.log1p(x_test)}, tf.log1p(y_test)))

        # We call repeat after shuffling,rather than before,to prevent separate epoches from blending together
        if training:
            dataset = dataset.shuffle(10 * batch_size, seed=RANDOM_SEED).repeat()

        dataset = dataset.batch(batch_size)
        iterator = dataset.make_one_shot_iterator()
        features, labels = iterator.get_next()
        return features, labels
    return _input_fn

In [16]:
_NUM_LAYERS_KEY = 'num_layers'


class _SimpleDNNBuilder(adanet.subnetwork.Builder):
    def __init__(self, optimizer, layer_size, num_layers, learning_mixture_weights, seed):
        self._optimizer = optimizer
        self._layer_size = layer_size
        self._num_layers = num_layers
        self._learn_mixture_weights = learning_mixture_weights
        self._seed = seed

    def build_subnetwork(self, features, logits_dimension, training, iteration_step,
                         summary, previous_ensemble=None):
        input_layer = tf.to_float(features[FEATURES_KEY])
        kernel_initializer = tf.glorot_uniform_initializer(seed=self._seed)
        last_layer = input_layer
        for _ in range(self._num_layers):
            last_layer = tf.layers.dense(last_layer, units=self._layer_size, activation=tf.nn.relu,
                                         kernel_initializer=kernel_initializer)
        logits = tf.layers.dense(last_layer, units=logits_dimension, kernel_initializer=kernel_initializer)

        persisted_tensors = {_NUM_LAYERS_KEY: tf.constant(self._num_layers)}
        return adanet.Subnetwork(last_layer=last_layer, logits=logits,
                                 complexity=self._measure_complexity(),
                                 persisted_tensors=persisted_tensors)

    def _measure_complexity(self):
        # Approximates Rademacher complexity as the square-root of the depth
        return tf.sqrt(tf.to_float(self._num_layers))

    def build_subnetwork_train_op(self, subnetwork, loss, var_list, labels,
                                  iteration_step, summary, previous_ensemble):
        return self._optimizer.minimize(loss=loss, var_list=var_list)

    def build_mixture_weights_train_op(self, loss, var_list, logits, labels,
                                       iteration_step, summary):
        if not self._learn_mixture_weights:
            return tf.no_op()
        return self._optimizer.minimize(loss=loss, var_list=var_list)

    @property
    def name(self):
        if self._num_layers == 0:
            # A DNN with no hidden layers is a linear model
            return 'linear'
        return '{}_layer_dnn'.format(self._num_layers)


class SimpleDNNGenerator(adanet.subnetwork.Generator):
    def __init__(self, optimizer, layer_size=32, learn_mixture_weights=False, seed=None):
        self._seed = seed
        self._dnn_builder_fn = functools.partial(_SimpleDNNBuilder, optimizer=optimizer, layer_size=layer_size
                                                 , learn_mixture_weights=learn_mixture_weights)

    def generate_candidates(self, previous_ensemble, iteration_number, previous_ensemble_reports, all_reports):
        num_layers = 0
        seed = self._seed
        if previous_ensemble:
            num_layers = tf.contrib.util.constant_value(previous_ensemble.weighted_subnetworks[-1].
                                                        subnetwork.persisted_tensor[_NUM_LAYERS_KEY])
        if seed is not None:
            seed += iteration_number
        return [self._dnn_builder_fn(num_layers=num_layers, seed=seed),
                self._dnn_builder_fn(num_layers=num_layers + 1, seed=seed)]

In [19]:
_NUM_LAYERS_KEY = "num_layers"


class _SimpleDNNBuilder(adanet.subnetwork.Builder):
  """Builds a DNN subnetwork for AdaNet."""

  def __init__(self, optimizer, layer_size, num_layers, learn_mixture_weights,
               seed):
    """Initializes a `_DNNBuilder`.

    Args:
      optimizer: An `Optimizer` instance for training both the subnetwork and
        the mixture weights.
      layer_size: The number of nodes to output at each hidden layer.
      num_layers: The number of hidden layers.
      learn_mixture_weights: Whether to solve a learning problem to find the
        best mixture weights, or use their default value according to the
        mixture weight type. When `False`, the subnetworks will return a no_op
        for the mixture weight train op.
      seed: A random seed.

    Returns:
      An instance of `_SimpleDNNBuilder`.
    """

    self._optimizer = optimizer
    self._layer_size = layer_size
    self._num_layers = num_layers
    self._learn_mixture_weights = learn_mixture_weights
    self._seed = seed

  def build_subnetwork(self,
                       features,
                       logits_dimension,
                       training,
                       iteration_step,
                       summary,
                       previous_ensemble=None):
    """See `adanet.subnetwork.Builder`."""

    input_layer = tf.to_float(features[FEATURES_KEY])
    kernel_initializer = tf.glorot_uniform_initializer(seed=self._seed)
    last_layer = input_layer
    for _ in range(self._num_layers):
      last_layer = tf.layers.dense(
          last_layer,
          units=self._layer_size,
          activation=tf.nn.relu,
          kernel_initializer=kernel_initializer)
    logits = tf.layers.dense(
        last_layer,
        units=logits_dimension,
        kernel_initializer=kernel_initializer)

    persisted_tensors = {_NUM_LAYERS_KEY: tf.constant(self._num_layers)}
    return adanet.Subnetwork(
        last_layer=last_layer,
        logits=logits,
        complexity=self._measure_complexity(),
        persisted_tensors=persisted_tensors)

  def _measure_complexity(self):
    """Approximates Rademacher complexity as the square-root of the depth."""
    return tf.sqrt(tf.to_float(self._num_layers))

  def build_subnetwork_train_op(self, subnetwork, loss, var_list, labels,
                                iteration_step, summary, previous_ensemble):
    """See `adanet.subnetwork.Builder`."""
    return self._optimizer.minimize(loss=loss, var_list=var_list)

  def build_mixture_weights_train_op(self, loss, var_list, logits, labels,
                                     iteration_step, summary):
    """See `adanet.subnetwork.Builder`."""

    if not self._learn_mixture_weights:
      return tf.no_op()
    return self._optimizer.minimize(loss=loss, var_list=var_list)

  @property
  def name(self):
    """See `adanet.subnetwork.Builder`."""

    if self._num_layers == 0:
      # A DNN with no hidden layers is a linear model.
      return "linear"
    return "{}_layer_dnn".format(self._num_layers)


class SimpleDNNGenerator(adanet.subnetwork.Generator):
  """Generates a two DNN subnetworks at each iteration.

  The first DNN has an identical shape to the most recently added subnetwork
  in `previous_ensemble`. The second has the same shape plus one more dense
  layer on top. This is similar to the adaptive network presented in Figure 2 of
  [Cortes et al. ICML 2017](https://arxiv.org/abs/1607.01097), without the
  connections to hidden layers of networks from previous iterations.
  """

  def __init__(self,
               optimizer,
               layer_size=32,
               learn_mixture_weights=False,
               seed=None):
    """Initializes a DNN `Generator`.

    Args:
      optimizer: An `Optimizer` instance for training both the subnetwork and
        the mixture weights.
      layer_size: Number of nodes in each hidden layer of the subnetwork
        candidates. Note that this parameter is ignored in a DNN with no hidden
        layers.
      learn_mixture_weights: Whether to solve a learning problem to find the
        best mixture weights, or use their default value according to the
        mixture weight type. When `False`, the subnetworks will return a no_op
        for the mixture weight train op.
      seed: A random seed.

    Returns:
      An instance of `Generator`.
    """

    self._seed = seed
    self._dnn_builder_fn = functools.partial(
        _SimpleDNNBuilder,
        optimizer=optimizer,
        layer_size=layer_size,
        learn_mixture_weights=learn_mixture_weights)

  def generate_candidates(self, previous_ensemble, iteration_number,
                          previous_ensemble_reports, all_reports):
    """See `adanet.subnetwork.Generator`."""

    num_layers = 0
    seed = self._seed
    if previous_ensemble:
      num_layers = tf.contrib.util.constant_value(
          previous_ensemble.weighted_subnetworks[
              -1].subnetwork.persisted_tensors[_NUM_LAYERS_KEY])
    if seed is not None:
      seed += iteration_number
    return [
        self._dnn_builder_fn(num_layers=num_layers, seed=seed),
        self._dnn_builder_fn(num_layers=num_layers + 1, seed=seed),
    ]

In [22]:
# @title AdaNet parameters
LEARNING_RATE = 0.001
TRAIN_STEPS = 100000
BATCH_SIZE = 32

LEARN_MIXTURE_WEIGHTS = False
ADANET_LAMBDA = 0
BOOSTING_ITERATIONS = 5

def train_and_evaluate(learn_mixture_weights=LEARN_MIXTURE_WEIGHTS, adanet_lambda=ADANET_LAMBDA):
    # Trains an adanet.Estimator to predict housing prices

    estimator = adanet.Estimator(
        # Since we are predicting housing prices,we will use a regression head that optimizes for MSE
        head=tf.contrib.estimator.regression_head(loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE),

        # Define the generator,which defines our search space of subnetworks to train as candidates to to add
        # to the final AdaNet model
        subnetwork_generator=SimpleDNNGenerator(optimizer=tf.train.RMSPropOptimizer(
            learning_rate=LEARNING_RATE), learn_mixture_weights=learn_mixture_weights,
            seed=RANDOM_SEED),

        # lambda is a the strength of complexity regularization.A larger value will penalize more complex subnetwprks
        adanet_lambda=adanet_lambda,

        # The number of train steps per iteration
        max_iteration_steps=TRAIN_STEPS // BOOSTING_ITERATIONS,

        # The evaluator will evaluate the model on the full training set to compute the overall adanet loss(
        # train loss + complexity regularization) to select the best candidate to include in the final adanet model
        evaluator=adanet.Evaluator(input_fn=input_fn('train', training=False, batch_size=BATCH_SIZE)),

        # Configuration for estimators
        config=tf.estimator.RunConfig(save_checkpoints_steps=50000, save_summary_steps=50000,
                                      tf_random_seed=RANDOM_SEED))

    # Train and evaluate using the tf.estimator tooling
    train_spec = tf.estimator.TrainSpec(input_fn=input_fn('train', training=True, batch_size=BATCH_SIZE),
                                        max_steps=TRAIN_STEPS)
    eval_spec = tf.estimator.EvalSpec(input_fn=input_fn('test', training=False, batch_size=BATCH_SIZE), steps=None)
    return tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

def ensemble_architecture(result):
    # Extract the ensemble architecture from evaluation results

    architecture = result["architecture/adanet/ensembles"]
    # The architecture is a serialized summary proto for temsorboard
    summary_proto = tf.summary.Summary.FromString(architecture)
    return summary_proto.value[0].tensor.string_val[0]

results, _ = train_and_evaluate()
print('Loss:', results['average_loss'])
print('Architecture:', ensemble_architecture(results))

INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_is_chief': True, '_num_worker_replicas': 1, '_save_checkpoints_secs': None, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_evaluation_master': '', '_master': '', '_service': None, '_model_dir': 'C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\tmplonjk62_', '_save_summary_steps': 50000, '_tf_random_seed': 42, '_task_id': 0, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000001285EC50>, '_device_fn': None, '_task_type': 'worker', '_train_distribute': None, '_save_checkpoints_steps': 50000, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None}


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 50000 or save_checkpoints_secs None.


INFO:tensorflow:Beginning training AdaNet iteration 0


INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:Finished evaluation at 2018-11-18-11:48:57


INFO:tensorflow:Saving dict for global step 100000: average_loss = 0.031948246, average_loss/adanet/adanet_weighted_ensemble = 0.031948246, average_loss/adanet/subnetwork = 0.041267823, average_loss/adanet/uniform_average_ensemble = 0.031948246, global_step = 100000, label/mean = 3.1049454, label/mean/adanet/adanet_weighted_ensemble = 3.1049454, label/mean/adanet/subnetwork = 3.1049454, label/mean/adanet/uniform_average_ensemble = 3.1049454, loss = 0.038237095, loss/adanet/adanet_weighted_ensemble = 0.038237095, loss/adanet/subnetwork = 0.04582113, loss/adanet/uniform_average_ensemble = 0.038237102, prediction/mean = 3.13776, prediction/mean/adanet/adanet_weighted_ensemble = 3.13776, prediction/mean/adanet/subnetwork = 3.1456003, prediction/mean/adanet/uniform_average_ensemble = 3.13776


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100000: C:\Users\ADMINI~1\AppData\Local\Temp\tmplonjk62_\model.ckpt-100000


INFO:tensorflow:Loss for final step: 0.0051082023.


Loss: 0.031948246
Architecture: b"| b'1_layer_dnn' | b'2_layer_dnn' | b'2_layer_dnn' | b'3_layer_dnn' | b'3_layer_dnn' |"


In [18]:
results, _ = train_and_evaluate(learn_mixture_weights=True)
print("Loss:", results["average_loss"])
print("Uniform average loss:", results["average_loss/adanet/uniform_average_ensemble"])
print("Architecture:", ensemble_architecture(results))


INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_is_chief': True, '_num_worker_replicas': 1, '_save_checkpoints_secs': None, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_evaluation_master': '', '_master': '', '_service': None, '_model_dir': 'C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\tmpuv725ui3', '_save_summary_steps': 50000, '_tf_random_seed': 42, '_task_id': 0, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000C8957B8>, '_device_fn': None, '_task_type': 'worker', '_train_distribute': None, '_save_checkpoints_steps': 50000, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None}


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 50000 or save_checkpoints_secs None.


INFO:tensorflow:Beginning training AdaNet iteration 0


INFO:tensorflow:Calling model_fn.


TypeError: __init__() got an unexpected keyword argument 'learn_mixture_weights'

In [ ]:
results, _ = train_and_evaluate(learn_mixture_weights=True, adanet_lambda=.015)
print("Loss:", results["average_loss"])
print("Uniform average loss:", results["average_loss/adanet/uniform_average_ensemble"])
print("Architecture:", ensemble_architecture(results))